# Ejercicio de optimización MVC

## Introducción

Este notebook contiene el ejercicio de optimización de la entrega.

En él se adaptan los ejemplos mostrados en el notebook proporcionado llamado "Ising y MaxCut con D-Wave", modificándolos para resolver el problema de Minimum Vertex Cover o MVC.

## Problema MVC

El problema elegido para su modelado y resolución es el de Minimum Vertex Cover, del cual incluyo una pequeña descripción por comodidad.

Partiendo de un grafo no dirigido, este problema consiste en encontrar un conjunto de vértices tan pequeño como sea posible, pero que contenga al menos un vértice de cada eje existente.

## Enfoque para la parametrización del problema

Para la solución de este problema, he utilizado un grafo simple arbitrario y calculado manualmente h y J a partir de él basándome en las explicaciones encontradas en el PDF proporcionado con varios ejemplos de problemas QUBO.

El grafo utilizado es el siguiente:

<img src="mvc-graph.png" width="30%">

Por consistencia con el método descrito en dicho PDF, he utilizado una constante de penalización P = 8.

Teniendo esto en cuenta, J y h obtienen los siguientes valores:

In [1]:
P = 8

J = { \
     (0,1):P, (0,2):P, (0,3):P, \
     (2,5):P, (3,5):P, (4,5):P, \
     (2,4):P \
}

h = { \
     0:1-3*P, 1:1-1*P, 2:1-3*P, \
     3:1-2*P, 4:1-2*P, 5:1-3*P \
}

Una vez definidos estos, me he abstraído del hamiltoniano generado, aunque he estudiado previamente el ejemplo mostrado en el PDF para asegurarme de que entiendo el problema y su solución. 

<img src="mvc-hamiltonian.png" width="100%">

Para continuar, simplemente utilizo los métodos encontrados en el notebook mencionado, aprovechando la disponibilidad de herramientas adecuadas dentro de la libreria dimod para resolver este tipo de problemas.

## Resolución Exacta

In [6]:
import numpy as np
import dimod

model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.BINARY)

print(model)

from dimod.reference.samplers import ExactSolver

sampler = ExactSolver()
solution = sampler.sample(model)

print(solution)

BinaryQuadraticModel({0: -23, 1: -7, 2: -23, 3: -15, 4: -15, 5: -23}, {(0, 1): 8, (0, 2): 8, (0, 3): 8, (2, 5): 8, (2, 4): 8, (3, 5): 8, (4, 5): 8}, 0.0, Vartype.BINARY)
    0  1  2  3  4  5 energy num_oc.
33  1  0  0  0  1  1  -53.0       1
57  1  0  1  0  0  1  -53.0       1
20  0  1  1  1  1  0  -52.0       1
22  1  0  1  1  1  0  -52.0       1
34  1  1  0  0  1  1  -52.0       1
38  1  0  1  0  1  1  -52.0       1
46  1  0  0  1  1  1  -52.0       1
52  0  1  1  1  0  1  -52.0       1
54  1  0  1  1  0  1  -52.0       1
58  1  1  1  0  0  1  -52.0       1
21  1  1  1  1  1  0  -51.0       1
37  1  1  1  0  1  1  -51.0       1
41  1  0  1  1  1  1  -51.0       1
43  0  1  1  1  1  1  -51.0       1
45  1  1  0  1  1  1  -51.0       1
53  1  1  1  1  0  1  -51.0       1
42  1  1  1  1  1  1  -50.0       1
62  1  0  0  0  0  1  -46.0       1
9   1  0  1  1  0  0  -45.0       1
11  0  1  1  1  0  0  -45.0       1
17  1  0  0  1  1  0  -45.0       1
23  0  0  1  1  1  0  -45.0       1
25

## Resolución con Simulated Annealing
En este caso, utilizamos un método heurístico que emula la computación cuántica adiabática en ordenadores clásicos.

Además de permitir unificar computación clásica y cuántica con un mismo modelo, tengo entendido que este método también ha permitido mejoras de rendimiento con ordenadores clásicos para una cantidad considerable de problemas.

In [7]:
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample(model, num_reads=10)
print(response)

   0  1  2  3  4  5 energy num_oc.
0  1  0  1  0  0  1  -53.0       1
1  1  0  1  0  0  1  -53.0       1
2  1  0  0  0  1  1  -53.0       1
3  1  0  1  0  0  1  -53.0       1
4  1  0  0  0  1  1  -53.0       1
5  1  0  1  0  0  1  -53.0       1
6  1  0  0  0  1  1  -53.0       1
7  1  0  1  0  0  1  -53.0       1
8  1  0  0  0  1  1  -53.0       1
9  1  0  1  0  0  1  -53.0       1
['BINARY', 10 rows, 10 samples, 6 variables]


## Resolución con Computación Cuántica Adiabática en servidor D-Wave
Requiere registro online en https://cloud.dwavesys.com/leap/

In [8]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample(model, num_reads=5000)
print(response)

ValueError: API token not defined